### **Clustering for Product Matching**

### Importing packages

In [125]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
import re

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Loading the data

In [126]:
data = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/unmatched_stockist_transactions.csv')
data.head()

,product_name,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,laibuta foliar feeds,murphy foliar feed,0.68,laib,eabl,0.22
1,White Candles - Riva,white line - mint & coriander,0.49,Halar Industries Ltd,zaam industries ltd.,0.83
2,Ampiclo-dawa dry syrup,vitastar pet syrup,0.40,.,mea ltd.,0.20
3,carvedilol(vidol)6.25mg,clopidol,0.45,Cosmos ltd,cosmos ltd.,0.91
4,Amoxil 'O'125mg,mineral 1-5725,0.41,GSK,wellstock,0.31


In [127]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8614931 entries, 0 to 8614930
Data columns (total 6 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   product_name              object 
 1   best_product_match        object 
 2   product_match_score       float64
 3   manufacturer_name         object 
 4   best_manufacturer_match   object 
 5   manufacturer_match_score  float64
dtypes: float64(2), object(4)
memory usage: 394.4+ MB


In [128]:
data.isna().sum()

product_name                    1
best_product_match              0
product_match_score             0
manufacturer_name           86649
best_manufacturer_match     83377
manufacturer_match_score        0
dtype: int64

### Data Preprocessing

In [129]:
# dropping null values in product_name column
data = data.dropna(subset = ['product_name'])

# cleaning the product name column
data['product_name'] = data['product_name'].apply(lambda x: x.strip())

# changing all strings to lowercase
data = data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
data.head()

,product_name,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,laibuta foliar feeds,murphy foliar feed,0.68,laib,eabl,0.22
1,white candles - riva,white line - mint & coriander,0.49,halar industries ltd,zaam industries ltd.,0.83
2,ampiclo-dawa dry syrup,vitastar pet syrup,0.40,.,mea ltd.,0.20
3,carvedilol(vidol)6.25mg,clopidol,0.45,cosmos ltd,cosmos ltd.,0.91
4,amoxil 'o'125mg,mineral 1-5725,0.41,gsk,wellstock,0.31


In [130]:
data['product_name'].nunique()  

129071

In [6]:
# filtering for unique product names
unique_product_names = data['product_name'].unique()
len(unique_product_names)

129071

In [7]:
# vectorizing the product names
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(unique_product_names)

In [ ]:
# # choosing the k-value using the elbow method
# clusters = [1000, 2000, 3000, 4000, 5000, 6000, 7000]
# silhouette_avg = []

# for cluster in clusters:
#     kmeans = KMeans(n_clusters = cluster, random_state = 42)
#     kmeans.fit(tfidf_matrix)
#     cluster_labels = kmeans.labels_
    
#     silhouette_avg.append(silhouette_score(tfidf_matrix, cluster_labels))
    
# plt.plot(clusters, silhouette_avg, 'bx-')
# plt.xlabel('Values of K') 
# plt.ylabel('Silhouette score') 
# plt.title('Silhouette analysis For Optimal k')
# plt.show()

In [ ]:
# # clustering the vectorized data
# k_values = [400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600]
# ssd = []

# for k in k_values:
#     kmeans = KMeans(n_clusters=k, random_state=0).fit(tfidf_matrix)
#     ssd.append(kmeans.inertia_)

# # plotting the elbow curve
# plt.plot(k_values, ssd, 'bx-')
# plt.xlabel('No of clusters (k)')
# plt.ylabel('inertia')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()

In [8]:
# clustering the data
num_clusters = 10000
kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(tfidf_matrix)
labels = kmeans.labels_

# creating a dataframe of the clusters
cluster_to_name = {}
for label in set(labels):
    indices = np.where(labels == label)[0]
    names = unique_product_names[indices].tolist()
    cluster_to_name[label] = names
    
# viewing the clusters
cluster_4_df = pd.DataFrame.from_dict(cluster_to_name.items())
cluster_4_df.rename(columns={0: 'cluster_id', 1: 'product_names'}, inplace=True)
cluster_4_df.set_index('cluster_id', inplace=True)
pd.set_option('display.max_colwidth', None)
cluster_4_df.head(5)

,product_names
cluster_id,
0,"[nozzle four holes, nozzle (4 holes), four holed nozzle]"
1,"[ord.nails 2"", ord.nails 3"", ord nails 5"", ord nails 3"", ord nails 2"", ord nails 4"", 3"" ord.nails 1/4, 1""ord. nails 1/4, ord. nails 2"", ord. nails 3"", ord. nails 4"", 1½'' ord. nails 1/4, 4"" ord. nails 1/4, 2"" ord. nails 1/4, 5"" ord. nails 1/4, 6"" ord. nails 1/4, ord.nails 4"", ord. nails 2 ½"", ord nails 6"", ord nails 2 1/2"", ord nails, ord nails 21/2""]"
2,"[kajamba growers, kajamba growers mash, growers kajamba]"
3,"[water heater black small, water heater black big, big black heater, water heater black]"
4,"[pliers large, pliers large]"


In [104]:
unique_names_df = pd.DataFrame({'product_name': unique_product_names,
                                'label': labels})
unique_names_df.head()
len(unique_names_df)

129071

### Topic modeling (1)

In [120]:
# extracting most common words from each cluster in order
cluster_word_freq = {}

for doc, cluster_label in zip(unique_product_names, labels):
    words = re.split(r'\s+|-|\(|\)|/|\\|\||,', doc)
    for word in words:
        if cluster_label in cluster_word_freq:
            cluster_word_freq[cluster_label][word] = cluster_word_freq[cluster_label].get(word, 0) + 1
        else:
            cluster_word_freq[cluster_label] = {word: 1}
    
for cluster_label in cluster_word_freq:
    cluster_word_freq[cluster_label] = sorted(cluster_word_freq[cluster_label].items(), key=lambda x: x[1], reverse=True)
    
cluster_word_freq_df = pd.DataFrame.from_dict(cluster_word_freq.items())
cluster_word_freq_df.rename(columns={0: 'cluster_id', 1: 'word_freq'}, inplace=True)
cluster_word_freq_df.head()

,cluster_id,word_freq
0,5598,"[(laibuta, 7), (foliar, 7), (feed, 3), (feeds, 2), (, 2), (liquid, 1)]"
1,1253,"[(, 18), (candles, 9), (riva, 9), (white, 2), (yellow, 2), (red, 2), (black, 1), (maroon, 1), (green, 1)]"
2,8257,"[(ampiclo, 16), (dawa, 14), (, 5), (ampicilin, 3), (dry, 2), (syrup, 2), (caps, 2), (neonatal, 1), (500mg, 1), (500, 1), (suspension, 1), (250mg, 1), (5ml, 1), (capsules, 1), (0.6ml, 1), (90mg, 1), (0.6, 1)]"
3,2806,"[(vidol, 8), (carvedilol, 6), (, 5), (6.25mg, 4), (tabs, 2), (28's, 1), (6.25mgs, 1), (12.5mgs, 1)]"
4,8278,"[(amoxil, 16), (, 4), (syrup, 3), (500mg, 2), ('o'125mg, 1), (syrp, 1), (amoxycillin, 1), (60ml, 1), (cap, 1), (500, 1), (100ml, 1), (capsules, 1), (dt, 1), (caps, 1), (dispersible, 1), (suspension, 1), (original, 1), (60mls, 1)]"


In [131]:
cluster_word_freq_df['cluster_name'] = cluster_word_freq_df['word_freq'].apply(lambda x: ' '.join(word[0] for word in x[:3]))
cluster_word_freq_df

,cluster_id,word_freq,cluster_name
0,5598,"[(laibuta, 7), (foliar, 7), (feed, 3), (feeds, 2), (, 2), (liquid, 1)]",laibuta foliar feed
1,1253,"[(, 18), (candles, 9), (riva, 9), (white, 2), (yellow, 2), (red, 2), (black, 1), (maroon, 1), (green, 1)]",candles riva
2,8257,"[(ampiclo, 16), (dawa, 14), (, 5), (ampicilin, 3), (dry, 2), (syrup, 2), (caps, 2), (neonatal, 1), (500mg, 1), (500, 1), (suspension, 1), (250mg, 1), (5ml, 1), (capsules, 1), (0.6ml, 1), (90mg, 1), (0.6, 1)]",ampiclo dawa
3,2806,"[(vidol, 8), (carvedilol, 6), (, 5), (6.25mg, 4), (tabs, 2), (28's, 1), (6.25mgs, 1), (12.5mgs, 1)]",vidol carvedilol
4,8278,"[(amoxil, 16), (, 4), (syrup, 3), (500mg, 2), ('o'125mg, 1), (syrp, 1), (amoxycillin, 1), (60ml, 1), (cap, 1), (500, 1), (100ml, 1), (capsules, 1), (dt, 1), (caps, 1), (dispersible, 1), (suspension, 1), (original, 1), (60mls, 1)]",amoxil syrup
...,...,...,...
9995,1745,"[(, 2), (4.0, 1), (single, 1), (core, 1), (red, 1), (black, 1), (yellow, 1)]",4.0 single
9996,6674,"[(fuel, 2), (filter, 2), (perkins, 2), (short, 1)]",fuel filter perkins
9997,7901,"[(copem, 2), (simlaw, 1)]",copem simlaw
9998,3288,"[(bestphos, 1), (5kg, 1)]",bestphos 5kg


In [ ]:
cluster_merge = cluster_word_freq_df[['cluster_id', 'cluster_name']]
cluster_merge = cluster_merge.rename(columns={'cluster_id': 'label'})

data = data.merge(unique_names_df, on='product_name', how='left')
data = data.merge(cluster_merge, on='label', how='left')

columns = data.columns.tolist()
columns.remove('cluster_name')
columns.insert(1, 'cluster_name')
data = data[columns]

data[:50]

In [137]:
data[data['product_match_score'] > 0.75].head()

,product_name,cluster_name,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
87445,laibuta kaolin,laibuta dcp kaolin,light kaolin,0.77,laibuta chemicals ltd,laibuta chemicals ltd.,0.95,661
87446,touchdown forteviot 500,syrup tabs,touchdown forte,0.77,syngenta,syngenta,0.94,88
87447,amigad,amigad 20g wgd,amigold,0.77,farmfix africa co. ltd.far,farmfix africa co. ltd.,0.92,3597
87448,farmate sprayer 20ltr,farmate sprayer pressure,farmate knapsack sprayer (20lt),0.77,muddo agrochemicals ltd,united agrochemicals ltd.,0.82,670
87449,dudumectin,syrup tabs,dudu acelamectin,0.77,guf,nutrifarm,0.31,88


In [135]:
# data.to_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/clustered_stockist_transactions.csv', index=False)

### Topic modeling (2)

In [10]:
# creating vectorizers for each cluster
vectorizers = []

for k in range(0, num_clusters):
    vectorizers.append(CountVectorizer(lowercase=True))
    
vectorizers[0]

CountVectorizer()

In [11]:
# vectorizing the data in each cluster
vectorized_data = []
fitted_vectorizers = []

for i, vectorizer in enumerate(vectorizers):
    indices = np.where(labels == i)[0]
    try:
        data = vectorizer.fit_transform(unique_product_names[indices])
        vectorized_data.append(data)
        fitted_vectorizers.append(vectorizer)
    except:
        print("Not enough instances in cluster {}".format(i))
        vectorized_data.append(None)

In [12]:
# getting topics per cluster
num_topics_per_cluster = 1

lda_models = []

for n in range(0, num_clusters):
    lda = LatentDirichletAllocation(n_components=num_topics_per_cluster, max_iter=10, 
                                    learning_method='online', verbose=False, random_state=0)
    lda_models.append(lda)
    
lda_models[0]

LatentDirichletAllocation(learning_method='online', n_components=1,
                          random_state=0, verbose=False)

In [13]:
# fitting the lda models
cluster_lda_data = []

for l, lda in enumerate(lda_models):
    print(f'Current cluster: {l}')
    
    if vectorized_data[l] != None:
        cluster_lda_data.append(lda.fit_transform(vectorized_data[l]))

Current cluster: 0
Current cluster: 1
Current cluster: 2
Current cluster: 3
Current cluster: 4
Current cluster: 5
Current cluster: 6
Current cluster: 7
Current cluster: 8
Current cluster: 9
Current cluster: 10
Current cluster: 11
Current cluster: 12
Current cluster: 13
Current cluster: 14
Current cluster: 15
Current cluster: 16
Current cluster: 17
Current cluster: 18
Current cluster: 19
Current cluster: 20
Current cluster: 21
Current cluster: 22
Current cluster: 23
Current cluster: 24
Current cluster: 25
Current cluster: 26
Current cluster: 27
Current cluster: 28
Current cluster: 29
Current cluster: 30
Current cluster: 31
Current cluster: 32
Current cluster: 33
Current cluster: 34
Current cluster: 35
Current cluster: 36
Current cluster: 37
Current cluster: 38
Current cluster: 39
Current cluster: 40
Current cluster: 41
Current cluster: 42
Current cluster: 43
Current cluster: 44
Current cluster: 45
Current cluster: 46
Current cluster: 47
Current cluster: 48
Current cluster: 49
Current cl

In [14]:
def selected_topics(model, vectorizer, top_n=4):
    current_words = []
    keywords = []

    for idx, topic in enumerate(model.components_):
        words = [(vectorizer.get_feature_names_out()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]]
        for word in words:
            if word[0] not in current_words:
                keywords.append(word)
                current_words.append(word[0])

    keywords.sort(key=lambda x: x[1], reverse=True)
    return_values = [word[0] for word in keywords]

    return return_values


all_keywords = []

for current_vectorizer, lda in enumerate(lda_models):
    print(f"Current Cluster: {current_vectorizer}")
    
    if vectorized_data[current_vectorizer] != None:
        all_keywords.append(selected_topics(lda, fitted_vectorizers[current_vectorizer]))

Current Cluster: 0
Current Cluster: 1
Current Cluster: 2
Current Cluster: 3
Current Cluster: 4
Current Cluster: 5
Current Cluster: 6
Current Cluster: 7
Current Cluster: 8
Current Cluster: 9
Current Cluster: 10
Current Cluster: 11
Current Cluster: 12
Current Cluster: 13
Current Cluster: 14
Current Cluster: 15
Current Cluster: 16
Current Cluster: 17
Current Cluster: 18
Current Cluster: 19
Current Cluster: 20
Current Cluster: 21
Current Cluster: 22
Current Cluster: 23
Current Cluster: 24
Current Cluster: 25
Current Cluster: 26
Current Cluster: 27
Current Cluster: 28
Current Cluster: 29
Current Cluster: 30
Current Cluster: 31
Current Cluster: 32
Current Cluster: 33
Current Cluster: 34
Current Cluster: 35
Current Cluster: 36
Current Cluster: 37
Current Cluster: 38
Current Cluster: 39
Current Cluster: 40
Current Cluster: 41
Current Cluster: 42
Current Cluster: 43
Current Cluster: 44
Current Cluster: 45
Current Cluster: 46
Current Cluster: 47
Current Cluster: 48
Current Cluster: 49
Current Cl

In [15]:
keywords_df = pd.DataFrame(all_keywords)
keywords_df.fillna('', inplace=True)
keywords_df['concat_name'] = keywords_df[0] + ' ' + keywords_df[1] + ' ' + keywords_df[2] + ' ' + keywords_df[3]
keywords_df.head()

,0,1,2,3,concat_name
0,nozzle,holes,four,holed,nozzle holes four holed
1,nails,ord,1½,21,nails ord 1½ 21
2,growers,kajamba,mash,,growers kajamba mash
3,heater,black,water,big,heater black water big
4,large,pliers,,,large pliers
